# Making adjustments to the constraint system

In the [previous example about model
adjustments](02b-model-modifications.md), we noted that some constraint
systems may be to complex to be changed within the limits of the usual FBC
model view, and we may require a sharper tool to do the changes we need. This
example shows how to do that by modifying the constraint systems that are
generated within COBREXA to represent the metabolic model contents.

## Background: Model-to-optimizer pipeline

## Background: Constraint trees

## Changing the model-to-optimizer pipeline

TODO clean up the stuff below:

In [1]:
using COBREXA

download_model(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
    "7bedec10576cfe935b19218dc881f3fb14f890a1871448fc19a9b4ee15b448d8",
)

import JSONFBCModels
import GLPK

model = load_model("e_coli_core.json")

[ Info: using cached `e_coli_core.json'


JSONFBCModels.JSONFBCModel(#= 95 reactions, 72 metabolites =#)

## Customizing the model

We can also modify the model. The most explicit way to do this is
to make a new constraint tree representation of the model.

In [2]:
import ConstraintTrees as C

ctmodel = flux_balance_constraints(model)

fermentation = ctmodel.fluxes.EX_ac_e.value + ctmodel.fluxes.EX_etoh_e.value

forced_mixed_fermentation =
    ctmodel * :fermentation^C.Constraint(fermentation, (10.0, 1000.0)) # new modified model is created

vt = optimized_values(
    forced_mixed_fermentation,
    objective = forced_mixed_fermentation.objective.value,
    optimizer = GLPK.Optimizer,
)

ConstraintTrees.Tree{Float64} with 4 elements:
  :fermentation       => 10.0
  :flux_stoichiometry => ConstraintTrees.Tree{Float64}(#= 72 elements =#)
  :fluxes             => ConstraintTrees.Tree{Float64}(#= 95 elements =#)
  :objective          => 0.633738

Models that cannot be solved return `nothing`. In the example below, the
underlying model is modified.

In [3]:
ctmodel.fluxes.ATPM.bound = C.Between(1000.0, 10000.0)

#TODO explicitly show here how false sharing looks like

vt = optimized_values(
    ctmodel,
    objective = ctmodel.objective.value,
    optimizer = GLPK.Optimizer,
)

Models can also be piped into the analysis functions

In [4]:
ctmodel.fluxes.ATPM.bound = C.Between(8.39, 10000.0) # revert
vt = optimized_values(
    ctmodel,
    objective = ctmodel.objective.value,
    optimizer = GLPK.Optimizer,
)

ConstraintTrees.Tree{Float64} with 3 elements:
  :flux_stoichiometry => ConstraintTrees.Tree{Float64}(#= 72 elements =#)
  :fluxes             => ConstraintTrees.Tree{Float64}(#= 95 elements =#)
  :objective          => 0.873922

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*